In [36]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score,accuracy_score,recall_score,f1_score
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [37]:
df=pd.read_csv('/Users/rocky/Desktop/mlops_projects/Mini Project/mlops-mini-project/data/external/tweet_emotions.csv')

In [38]:
df.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [39]:
df.drop(['tweet_id'], axis=1, inplace=True)


In [40]:
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [41]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# -----------------------------
# DOWNLOAD NLTK DATA
# -----------------------------
nltk.download("stopwords", quiet=True)
nltk.download("wordnet", quiet=True)

# -----------------------------
# GLOBAL OBJECTS
# -----------------------------
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

# -----------------------------
# TEXT CLEANING FUNCTIONS
# -----------------------------
def lemmatization(text):
    return " ".join(lemmatizer.lemmatize(w) for w in text.split())

def remove_stopword(text):
    return " ".join(w for w in text.split() if w not in stop_words)

def remove_digit(text):
    return " ".join(w for w in text.split() if not w.isdigit())

def lower_case(text):
    return text.lower()

def removing_punctuations(text):
    text = re.sub(r'[!"#$%&\'()*+,./:;<=>?@[\]^_`{|}~]', " ", text)
    return re.sub(r"\s+", " ", text).strip()

def removing_urls(text):
    return re.sub(r"https?://\S+|www\.\S+", "", text)

def normalize_text(df):
    """Apply sequential text cleaning steps."""
    
    if "content" not in df.columns:
        raise KeyError("Input dataframe must contain a 'content' column.")
    
    df["content"] = (
        df["content"]
        .astype(str)
        .apply(lower_case)
        .apply(removing_urls)
        .apply(removing_punctuations)
        .apply(remove_digit)
        .apply(remove_stopword)
        .apply(lemmatization)
    )

    return df




[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1032)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1032)>


In [42]:
df=normalize_text(df)

In [43]:
df.head()

,sentiment,content
0,empty,tiffanylue know listenin bad habit earlier sta...
1,sadness,layin n bed headache ughhhh waitin call
2,sadness,funeral ceremony gloomy friday
3,enthusiasm,want hang friend soon
4,neutral,dannycastillo want trade someone houston ticke...


In [44]:
x=df['sentiment'].isin(['happiness','sadness'])
df=df[x]

In [45]:
df.head()

,sentiment,content
1,sadness,layin n bed headache ughhhh waitin call
2,sadness,funeral ceremony gloomy friday
6,sadness,sleep im thinking old friend want married damn...
8,sadness,charviray charlene love miss
9,sadness,kelcouch sorry least friday


In [46]:
vectorizer=CountVectorizer(max_features=1000)
x=vectorizer.fit_transform(df['content'])
y=df['sentiment'].map({'sadness':0,'happiness':1})

In [47]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)

In [50]:
import dagshub
import mlflow

dagshub.init(repo_owner='Rocky0412', repo_name='mlops-mini-project', mlflow=True)
mlflow.set_tracking_uri('https://dagshub.com/Rocky0412/mlops-mini-project.mlflow')
mlflow.set_experiment('LogisticRegression BaseLine')

with mlflow.start_run() as run:
    mlflow.log_param('vectorizer','Bag of word')
    mlflow.log_param('max feature',1000)
    mlflow.log_param('test_size',0.2)

 
    import os
    notebook_path = 'exp1_baseline_model.ipynb'

    os.system(
    f'source venv/bin/activate && '
    f'jupyter nbconvert --to notebook --execute --inplace {notebook_path}')


    mlflow.log_artifact(notebook_path)

    model= LogisticRegression()
    model.fit(xtrain,ytrain)

    mlflow.log_param('model','Logistic Regression')

    #model Evalution
    y_pred=model.predict(xtest)
    mlflow.log_metric('accuracy',accuracy_score(ytest,y_pred))
    mlflow.log_metric('f1_score',f1_score(ytest,y_pred))
    mlflow.log_metric('precision',precision_score(ytest,y_pred))
    mlflow.log_metric('recall_score',recall_score(ytest,y_pred))

Initialized MLflow to track repo "Rocky0412/mlops-mini-project"

Repository Rocky0412/mlops-mini-project initialized!

sh: venv/bin/activate: No such file or directory


🏃 View run luxuriant-donkey-845 at: https://dagshub.com/Rocky0412/mlops-mini-project.mlflow/#/experiments/1/runs/bad72cf4f1d84cde80f093e62e08772a
🧪 View experiment at: https://dagshub.com/Rocky0412/mlops-mini-project.mlflow/#/experiments/1
